# Synonyms API quick start

<a target="_blank" href="https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/06-synonyms-api.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 대화형 노트북은 공식 [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html)를 사용하여 Synonyms API([blog post](https://www.elastic.co/blog/update-synonyms-elasticsearch-introducing-synonyms-api), [API documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/synonyms-apis.html))를 소개합니다. 동의어를 사용하면 유사한 의미를 가진 용어 간의 관계를 정의하여 검색 관련성을 높일 수 있습니다. 이 노트북에서는 동의어 집합을 생성 및 업데이트하고, 동의어를 사용하도록 인덱스를 구성하고, 관련성을 높이기 위해 동의어를 활용하는 쿼리를 실행합니다.

## Elasticsearch/Kibana 실행

먼저 Elasticsearch/Kibana를 실행하세요.

## 패키지 설치 및 모듈 가져오기

시작하려면 Python 클라이언트를 사용하여 Elastic 배포에 연결해야 합니다.

먼저 `elasticsearch` Python 클라이언트를 설치해야 합니다.

In [1]:
!pip install -qU elasticsearch

## Elasticsearch 클라이언트 초기화

이제 [Elasticsearch Python 클라이언트](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html) 인스턴스를 생성합니다.   
Eleasticsearch URL, 사용자 ID, 사용자 Password, 인증서 경로 정보가 필요하며, 인증서 경로는 (auto_install 시) `kibana.yml` 하단에 기재되어 있습니다.
Elasticsearch에 로컬로 연결하는 방법에 대해  [자세히 알아보세요](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later)

In [2]:
from elasticsearch import Elasticsearch
from getpass import getpass

ES_URL = input('Elasticsearch URL: ')
ES_ID = "elastic" 
ES_PW = getpass('elastic user PW: ')
CERT_PATH = input('Elasticsearch cer 파일 경로: ')

# Create the client instance
client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_ID, ES_PW),
    ca_certs=CERT_PATH
)

클라이언트가 이 테스트에 연결되었는지 확인합니다.

In [3]:
print(client.info())

{'name': 'DESKTOP-2AKM6J7', 'cluster_name': 'elasticsearch', 'cluster_uuid': '5tEVK4EEQ0WPUotyTCvXYg', 'version': {'number': '8.11.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '6f9ff581fbcde658e6f69d6ce03050f060d1fd0c', 'build_date': '2023-11-11T10:05:59.421038163Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## 일부 테스트 데이터 색인화

클라이언트가 설정되어 Elastic 배포에 연결되었습니다.
이제 Elasticsearch 쿼리의 기본 사항을 테스트하려면 일부 데이터가 필요합니다.
다음 필드와 함께 작은 제품 색인을 사용합니다.:

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### 동의어 세트 만들기

먼저 초기 동의어 세트를 만들어 보겠습니다.

In [4]:
synonyms_set = [
    {
        "id": "synonym-1",
        "synonyms": "js, javascript, java script"
    }
]

client.synonyms.put_synonym(id="my-synonyms-set", synonyms_set=synonyms_set)

ObjectApiResponse({'result': 'created', 'reload_analyzers_details': {'_shards': {'total': 24, 'successful': 18, 'failed': 0}, 'reload_details': []}})

### 인덱스 구성

`book_index`라는 이름으로 이전에 생성된 색인이 없는지 확인하세요.

In [5]:
client.indices.delete(index="book_index", ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})

🔐 참고: 언제든지 이 섹션으로 돌아와 위의 `삭제` 기능을 실행하여 색인을 제거하고 처음부터 시작할 수 있습니다.



In order to use synonyms, we need to define a [custom analyzer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-custom-analyzer.html) that uses the [`synonym`](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-tokenfilter.html) or [`synonym_graph`](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-graph-tokenfilter.html) token filter. Let's create an index that's configured to use an appropriate custom analyzer.


동의어를 사용하려면 [`synonym`](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-tokenfilter.html) 또는 [`synonym_graph`](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-graph-tokenfilter.html) 토큰 필터를 사용하는 [custom analyzer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-custom-analyzer.html) that uses the [`synonym`](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-tokenfilter.html)를 정의해야 합니다. 적절한 custom analyzer를 사용하도록 구성된 인덱스를 만들어 보겠습니다.

In [6]:
settings = {
    "analysis": {
        "analyzer": {
            "my_custom_index_analyzer": {
                "tokenizer": "standard",
                "filter": [
                    "lowercase"
                ]
            },
            "my_custom_search_analyzer": {
                "tokenizer": "standard",
                "filter": [
                    "lowercase",
                    "my_synonym_filter"
                ]
            }
        },
        "filter": {
            "my_synonym_filter": {
                "type": "synonym_graph",
                "synonyms_set": "my-synonyms-set",
                "updateable": True
            }
        }
    }
}

mappings = {
    "properties": {
        "title": {
            "type": "text",
            "analyzer": "my_custom_index_analyzer",
            "search_analyzer": "my_custom_search_analyzer"
        },
        "summary": {
            "type": "text",
            "analyzer": "my_custom_index_analyzer",
            "search_analyzer": "my_custom_search_analyzer"
        }
    }
}

client.indices.create(index='book_index', mappings=mappings, settings=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'book_index'})

구성에서 주의해야 할 몇 가지 사항이 있습니다.

- 우리는 [`synonym_graph` token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-graph-tokenfilter.html)를 사용하고 있습니다.
- `my_custom_index_analyzer`와 `my_custom_search_analyzer`라는 두 가지 분석기를 정의했습니다.` my_custom_search_analyzer`는 [search analyzer](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-analyzer.html)로 사용됩니다.
- `my_synonym_filter`는 `my_custom_search_analyzer`에서만 사용됩니다.

Synony_graph 토큰 필터를 사용하면 여러 단어로 된 동의어를 사용할 수 있습니다. 그러나 이 필터는 검색 시에만 적용하는 것이 중요하므로 my_custom_search_analyzer에서만 사용합니다. 그리고 동의어는 검색 시에만 적용되므로 다시 색인화하지 않고도 업데이트할 수 있습니다.

동의어 검색 시간에 대한 자세한 배경 정보는 [_The same, but different: Boosting the power of Elasticsearch with synonyms_](https://www.elastic.co/blog/boosting-the-power-of-elasticsearch-with-synonyms)를 참조하세요.

## 인덱스 채우기

다음 명령어를 실행하여 이 [dataset](https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json)의 인기 프로그래밍 서적에 대한 정보가 포함된 일부 테스트 데이터를 업로드하세요.

In [7]:
import json
import os

cwd = os.getcwd()
url = cwd + "/data.json"
response = open(url)
books = json.loads(response.read())

operations = []
for book in books:
    operations.append({"index": {"_index": "book_index"}})
    operations.append(book)
client.bulk(index="book_index", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 748, 'items': [{'index': {'_index': 'book_index', '_id': 'WsiDe4wBPi0nl_xGRxHI', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'W8iDe4wBPi0nl_xGRxHI', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'XMiDe4wBPi0nl_xGRxHI', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'XciDe4wBPi0nl_xGRxHI', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_

## 참고: Elasticsearch 검색 결과를 예쁘게 인쇄합니다.

`search` API 호출은 읽기 어려운 중첩 JSON을 반환합니다.
예제에서 사람이 읽을 수 있는 멋진 출력을 반환하기 위해 `pretty_search_response`라는 작은 함수를 만들겠습니다.

In [8]:
def pretty_search_response(response):
    if len(response['hits']['hits']) == 0:
        print('Your search returned no results.')
    else:
        for hit in response['hits']['hits']:
            id = hit['_id']
            publication_date = hit['_source']['publish_date']
            score = hit['_score']
            title = hit['_source']['title']
            summary = hit['_source']['summary']
            publisher = hit["_source"]["publisher"]
            num_reviews = hit["_source"]["num_reviews"]
            authors = hit["_source"]["authors"]
            pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}")
            print(pretty_output)

## 쿼리 실행

일부 Elasticsearch 쿼리에서 동의어를 사용해 보겠습니다. Javascript에 관한 책을 검색하는 것부터 시작하겠습니다.

In [9]:
response = client.search(
    index="book_index",
    query={
        "multi_match": {
            "query": "java script",
            "fields": [
                "title^10",
                "summary",
            ]
        }
    }
)

pretty_search_response(response)


ID: YMiDe4wBPi0nl_xGRxHI
Publication date: 2018-12-04
Title: Eloquent JavaScript
Summary: A modern introduction to programming
Publisher: no starch press
Reviews: 38
Authors: ['marijn haverbeke']
Score: 21.44283

ID: X8iDe4wBPi0nl_xGRxHI
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Publisher: oreilly
Reviews: 36
Authors: ['kyle simpson']
Score: 20.531933

ID: Y8iDe4wBPi0nl_xGRxHI
Publication date: 2008-05-15
Title: JavaScript: The Good Parts
Summary: A deep dive into the parts of JavaScript that are essential to writing maintainable code
Publisher: oreilly
Reviews: 51
Authors: ['douglas crockford']
Score: 17.985434


"java script"라는 용어를 검색했지만 "JS" 및 "JavaScript"라는 용어가 포함된 결과를 얻었습니다. 우리의 동의어가 작동하고 있습니다!

이제 AI에 관한 책을 검색해 보겠습니다.

In [10]:
response = client.search(
    index="book_index",
    query={
        "multi_match": {
            "query": "AI",
            "fields": [
                "title^10",
                "summary",
            ]
        }
    }
)

pretty_search_response(response)

Your search returned no results.


결과가 없습니다! 'artificial intelligence'이라는 용어를 사용하지만 'AI'라는 용어는 사용하지 않는 책도 있습니다. 이전 쿼리가 결과를 반환하도록 Synonyms API를 사용하여 "AI"에 대한 새 동의어 규칙을 추가해 보겠습니다.

In [11]:
client.synonyms.put_synonym_rule(set_id="my-synonyms-set", rule_id="synonym-2", synonyms="ai, artificial intelligence")

ObjectApiResponse({'result': 'created', 'reload_analyzers_details': {'_shards': {'total': 26, 'successful': 19, 'failed': 0}, 'reload_details': [{'index': 'book_index', 'reloaded_analyzers': ['my_custom_search_analyzer'], 'reloaded_node_ids': ['NPC20c3ORmO7ryEMjY7-OA']}]}})

쿼리를 다시 실행하면 이제 몇 가지 결과를 얻을 수 있습니다.

In [12]:
response = client.search(
    index="book_index",
    query={
        "multi_match": {
            "query": "AI",
            "fields": [
                "title^10",
                "summary",
            ]
        }
    }
)

pretty_search_response(response)


ID: XciDe4wBPi0nl_xGRxHI
Publication date: 2020-04-06
Title: Artificial Intelligence: A Modern Approach
Summary: Comprehensive introduction to the theory and practice of artificial intelligence
Publisher: pearson
Reviews: 39
Authors: ['stuart russell', 'peter norvig']
Score: 44.12741


## 결론

동의어 API를 사용하면 검색 색인에 사용되는 동의어를 실시간으로 동적으로 생성 및 수정할 수 있습니다. 이 노트를 읽고 나면 Synonyms API를 검색 환경에 통합하는 데 필요한 모든 것을 갖추게 될 것입니다!